In [0]:
def streamread(form,schema,path,**options):
    df=spark.readStream.format(form).schema(schema).options(**options).load(path)
    return df

In [0]:
ID	Name	Age	location


In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("ID", StringType(), True),
    StructField("Name", StringType(), True),
    StructField("Age", StringType(), True),
    StructField("location", StringType(), True)
])

In [0]:
def witestream(forma,mode,df,**options):
    Query=df.writeStream.format(forma).outputMode(mode).options(**options).start()
    Query.awaitTermination(timeout=60000)
    return Query

In [0]:
def writestream(forma, mode, df, **options):
    Query = df.writeStream \
        .format(forma) \
        .outputMode(mode) \
        .options(**options) \
        .start()
    Query.awaitTermination(timeout=60000)
    return Query

# Example usage


In [0]:
writestream('delta','update', df, path='dbfs:/FileStore/stream2/write/', checkpointLocation='dbfs:/FileStore/stream2/checkpoint')


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3897863891872452>:1
----> 1 writestream('delta','update', df, path='dbfs:/FileStore/stream2/write/', checkpointLocation='dbfs:/FileStore/stream2/checkpoint')

File <command-3897863891872451>:2, in writestream(forma, mode, df, **options)
      1 def writestream(forma, mode, df, **options):
----> 2     Query = df.writeStream \
      3         .format(forma) \
      4         .outputMode(mode) \
      5         .options(**options) \
      6         .start()
      7     Query.awaitTermination(timeout=60000)
      8     return Query

File /databricks/spark/python/pyspark/sql/streaming/readwriter.py:1385, in DataStreamWriter.start(self, path, format, outputMode, partitionBy, queryName, **options)
   1383     self.queryName(queryName)
   1384 if path is None:
-> 1385     return self._sq(self._jwrite.start())
   1386 else:
   1387

In [0]:
# xdbutils.fs.mkdirs('/FileStore/stream2/checkpoint')

Out[8]: True

In [0]:
form='csv'
options={'header':True}
path='dbfs:/FileStore/stream2/read/'
df=streamread(form,schema,path,header=True)
df.display()

ID,Name,Age,location
1,Hari,12,TN
2,Mari,15,HYD
3,Pari,14,KE
4,Nari,17,KA
5,Bari,18,PJ
6,Hara,16,KY
2,Mari,15,TN
4,Narayan,17,KA


In [0]:
writestream('delta', "update", df, path='dbfs:/FileStore/stream2/write/', checkpointLocation='dbfs:/FileStore/stream2/checkpoint')

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3897863891872442>:1
----> 1 writestream('delta', "update", df, path='dbfs:/FileStore/stream2/write/', checkpointLocation='dbfs:/FileStore/stream2/checkpoint')

File <command-3897863891872438>:2, in writestream(forma, mode, df, **options)
      1 def writestream(forma,mode,df,**options):
----> 2     Query=df.writeStream.format(forma).outputMode(mode).options(**options).start()
      3     Query.awaitTermination(timeout=60000)
      4     return Query

File /databricks/spark/python/pyspark/sql/streaming/readwriter.py:1385, in DataStreamWriter.start(self, path, format, outputMode, partitionBy, queryName, **options)
   1383     self.queryName(queryName)
   1384 if path is None:
-> 1385     return self._sq(self._jwrite.start())
   1386 else:
   1387     return self._sq(self._jwrite.start(path))

File /databricks/spark/python/li

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import lit,col,expr
from delta.tables import DeltaTable

In [0]:
path='dbfs:/FileStore/stream2/write/'

In [0]:
mergeAndSaveDf(df,path,'scd11','ID')

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3897863891872446>:1
----> 1 mergeAndSaveDf(df,path,'scd11','ID')

File <command-3897863891872443>:8, in mergeAndSaveDf(df, path, target_table_name, mergeCol)
      4     deltaTable.alias("target").merge(
      5       source=df.alias("source"),
      6       condition=f'target.{mergeCol}=source.{mergeCol}').whenMatchedUpdate().whenNotMatchedInsert().execute()
      7 else:
----> 8     writestream('delta', "update", df, path='dbfs:/FileStore/stream2/write/', checkpointLocation='dbfs:/FileStore/stream2/checkpoint')

File <command-3897863891872438>:2, in writestream(forma, mode, df, **options)
      1 def writestream(forma,mode,df,**options):
----> 2     Query=df.writeStream.format(forma).outputMode(mode).options(**options).start()
      3     Query.awaitTermination(timeout=60000)
      4     return Query

File /databricks/sp

In [0]:
def mergeAndSaveDf(df,path,target_table_name,mergeCol):
    if DeltaTable.isDeltaTable(spark,f"{path}"):
        deltaTable = DeltaTable.forPath(spark, f"{path}")
        deltaTable.alias("target").merge(
          source=df.alias("source"),
          condition=f'target.{mergeCol}=source.{mergeCol}').whenMatchedUpdate().whenNotMatchedInsert().execute()
    else:
        writestream('delta', "update", df, path='dbfs:/FileStore/stream2/write/', checkpointLocation='dbfs:/FileStore/stream2/checkpoint')

In [0]:
Query=df.writeStream.format(forma).outputMode(mode).options(**options).start()
    Query.awaitTermination(timeout=60000)

In [0]:
dbutils.fs.rm('dbfs:/FileStore/stream2/read/day1.xlsx')


Out[20]: True